# Лабораторная работа No3
## Задача
1. Возьмите данные, представленные в файле
`Data/Lab3/Steam_2024_bestRevenue_1500.csv`:

    Загрузите данные в вашу схему данных.  
    Сделайте нормализацию модели данных, выделив в отдельные таблицы следующие атрибуты из файла:
    - publishers (название компании, кто опубликовал игру)
    - publisherClass (класс игры)
    - developers (команда разработчиков игры)

    Полученные 4 таблицы должны быть связаны внешними ключами.

## Создаем подключение к БД

In [278]:
import csv
import psycopg2
from datetime import datetime
from tabulate import tabulate

### Подключение к PostgreSQL
connection = psycopg2.connect(
    host="localhost",
    port="5432",
    database="",
    user="",
    password="",
)
### Создание курсора
cursor = connection.cursor()

print("Подключение к PostgreSQL успешно.")

Подключение к PostgreSQL успешно.


## Создание схемы

In [279]:
cursor.execute("CREATE SCHEMA IF NOT EXISTS ivan_patakin;")
connection.commit()

print("Схема 'ivan_patakin' создана.")

Схема 'ivan_patakin' создана.


## Создание таблиц

In [280]:
cursor.execute("""
    CREATE TABLE ivan_patakin.publishers (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255) UNIQUE NOT NULL
    );
""")
connection.commit()

cursor.execute("""
    CREATE TABLE ivan_patakin.publisher_class (
        id SERIAL PRIMARY KEY,
        class VARCHAR(255) UNIQUE NOT NULL
    );
""")
connection.commit()

cursor.execute("""
    CREATE TABLE ivan_patakin.developers (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255) UNIQUE NOT NULL
    );
""")
connection.commit()

cursor.execute("""
    CREATE TABLE ivan_patakin.games (
        steam_id BIGINT PRIMARY KEY,
        title VARCHAR(255) NOT NULL,
        release_date DATE,
        copies_sold INT,
        price NUMERIC(10, 2),
        revenue NUMERIC(15, 2),
        avg_playtime NUMERIC(10, 2),
        review_score INT,
        publisher_id INT REFERENCES ivan_patakin.publishers(id),
        class_id INT REFERENCES ivan_patakin.publisher_class(id),
        developer_id INT REFERENCES ivan_patakin.developers(id)
    );
""")
connection.commit()

print("Таблицы созданы.")

Таблицы созданы.


## Загрузка данных из CSV

### Наполняем таблицы
- publishers
- publisher_class
- developers

In [281]:
unique_publishers = set()
unique_classes = set()
unique_developers = set()
games_data = []

with open('Data/Lab3/Steam_2024_bestRevenue_1500.csv', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        if row['publishers']:
            unique_publishers.add(row['publishers'])
        if row['publisherClass']:
            unique_classes.add(row['publisherClass'])
        if row['developers']:
            unique_developers.add(row['developers'])
        games_data.append(row)
        
# Вставляем уникальные данные в publishers
for publisher in unique_publishers:
    cursor.execute("""
    INSERT INTO ivan_patakin.publishers (name)
    VALUES (%s) ON CONFLICT (name) DO NOTHING;
    """, (publisher,))

connection.commit()
print("Таблица publishers заполнена")

# Вставляем уникальные данные в publisher_class
for publisher_class in unique_classes:
    cursor.execute("""
    INSERT INTO ivan_patakin.publisher_class (class)
    VALUES (%s) ON CONFLICT (class) DO NOTHING;
    """, (publisher_class,))

connection.commit()
print("Таблица publisher_class заполнена")

# Вставляем уникальные данные в developers
for developer in unique_developers:
    cursor.execute("""
    INSERT INTO ivan_patakin.developers (name)
    VALUES (%s) ON CONFLICT (name) DO NOTHING;
    """, (developer,))

connection.commit()
print("Таблица developers заполнена")


Таблица publishers заполнена
Таблица publisher_class заполнена
Таблица developers заполнена


### Теперь наполняем таблицу games

In [282]:
for row in games_data:
    # Получаем id для внешних ключей
    publisher_id = None
    if row['publishers']:
        cursor.execute("SELECT id FROM ivan_patakin.publishers WHERE name = %s;", (row['publishers'],))
        result = cursor.fetchone()
        if result:
            publisher_id = result[0]

    class_id = None
    if row['publisherClass']:
        cursor.execute("SELECT id FROM ivan_patakin.publisher_class WHERE class = %s;", (row['publisherClass'],))
        result = cursor.fetchone()
        if result:
            class_id = result[0]

    developer_id = None
    if row['developers']:
        cursor.execute("SELECT id FROM ivan_patakin.developers WHERE name = %s;", (row['developers'],))
        result = cursor.fetchone()
        if result:
            developer_id = result[0]

    # Вставляем данные в games
    cursor.execute("""
    INSERT INTO ivan_patakin.games (
        steam_id, title, release_date, copies_sold, price, revenue, avg_playtime, review_score,
        publisher_id, class_id, developer_id
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, (
        int(row['steamId']),
        row['name'],
        datetime.strptime(row['releaseDate'], '%d-%m-%Y').date(),
        int(row['copiesSold']),
        float(row['price']) if row['price'] else None,
        float(row['revenue']),
        float(row['avgPlaytime']),
        int(row['reviewScore']) if row['reviewScore'] else None,
        publisher_id,
        class_id,
        developer_id
    ))

connection.commit()

print("Данные успешно загружены.")

Данные успешно загружены.


## Проверка данных

In [283]:
print("Таблица publishers")
cursor.execute("""
    SELECT name FROM ivan_patakin.publishers
""")

rows = cursor.fetchall()
headers = ["Название"]
print(tabulate(rows, headers=headers))

Таблица publishers
Название
------------------------------------------------------------
Octo Games
Mammorize
HeatPot Games
Landfall
画域互动
Lokin
2 Left Thumbs
墨元素
Crytivo
Dear Villagers,Doyoyo Games
BLAMCAM Interactive
Mendoka
SinginGiant
2K Games
MaDDoG
Brute Farce
Green Pine Studios
Ronin SoftWork
Poking Water Games
Reborn Entertainment
Lizzycat / Lizabeth Gidlund
Early Morning Studio
PERFECT WORLD GAMES
Night Eyes Interactive
Farlight Games
Valkyrie Initiative
Joy Way
mino_dev
HIKARI FIELD
Clickteam
Tann
Mash Games
FooxiedGames
Bigmode
Rad Codex
Tenth Art Studio
Yogscast Games,Gamersky Games
Dynamight Studios
Kitsune Games,MidBoss, LLC.
Toonslab
iFAction Studio,BD Games
Pixel Forge Games
akhsotoen
WSS playground
Fireshine Games
Chinesegamer
Gaijin Network Ltd
Sven Ahlgrimm
BlackMark Studio,Valkyrie Initiative
QueseraGames Co., Ltd.
Whalestork Interactive
BeXide Inc.
Night Signal Entertainment
Kalypso Media
Soda Game Studio
Lorke Records, Inh. Anne Baumann
Frost Giant Studios
ACQUIRE 

In [284]:
print("Таблица publisher_class")
cursor.execute("""
    SELECT class FROM ivan_patakin.publisher_class
""")

rows = cursor.fetchall()
headers = ["Название"]
print(tabulate(rows, headers=headers))

Таблица publisher_class
Название
----------
AA
Indie
AAA
Hobbyist


In [285]:
print("Таблица developers")
cursor.execute("""
    SELECT name FROM ivan_patakin.developers
""")

rows = cursor.fetchall()
headers = ["Название"]
print(tabulate(rows, headers=headers))

Таблица developers
Название
----------------------------------------------------------------------
画域互动
Trialforge Studio
Mendoka
SinginGiant
Poking Water Games
Lizzycat / Lizabeth Gidlund
Mash Games
Rad Codex
Codemasters
BeXide Inc.
Frost Giant Studios
ACQUIRE Corp.
Doot,Blibloop
Pixel Ferrets
PixelCraze s.r.o.
Deathyell Games
helpnode
Ernestas Norvaišas
Team Ranger
Supergiant Games
Ewoud van der Werf,Nils Slijkerman
Casey Clyde
Blinkmoon Games
MLMEDIA
Artefacts Studio
AIROTS,非魚創意
Goblinz Studio
Inceton games
KomeFukuro
WAZAZAK
Narko Games
Cloudcade
L&P
Game-Labs
Don Pachi
MYSTERY CROWN
Cradle Games
Red Nexus Games Inc.
BanzaiProject
Nuci
Wx3 Labs, LLC
Bug Inventors
Kvltgames
Red Dot Studio
Furniture & Mattress LLC
Canteen
STAR ENGINE PROJECT
Microbird Games
やぶから堂,たまぷろじぇくと
4 Dimension Games
Tomas Sala
BRAWEA LTD
OKJOY
Scienart Games
Furlough Games
AO Games
Macrojoy
Catfood Studio
QUByte Interactive
Studio 397
Bullfrog Productions
101XP GAME STUDIOS
Incuvo
Caged Element Inc.
光禾文化
snekf

In [286]:
print("Таблица games")
cursor.execute("""
    SELECT * FROM ivan_patakin.games ORDER BY steam_id;
""")

rows = cursor.fetchall()
headers = ["steam_id", "Название", "Дата релиза", "Проданные копии", "Цена", "Доход", "Среднее время игры", "Оценка", "publisher_id", "class_id", "developer_id"]
print(tabulate(rows, headers=headers))

Таблица games
  steam_id  Название                                                                                                                                                                                                         Дата релиза      Проданные копии    Цена             Доход    Среднее время игры    Оценка    publisher_id    class_id    developer_id
----------  ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  -------------  -----------------  ------  ----------------  --------------------  --------  --------------  ----------  --------------
     24880  The Saboteur™                                                                                                                                                                                                    2024-03-07                 53376    9.99  290153     

## Создадим view для таблицы game

In [287]:
cursor.execute("""
    CREATE VIEW ivan_patakin.games_view AS
    SELECT 
        g.steam_id,
        g.title AS game_title,
        COALESCE(g.release_date::TEXT, '-') AS release_date,
        COALESCE(TO_CHAR(g.copies_sold, 'FM999,999,999.00'), '-') AS copies_sold,
        COALESCE(TO_CHAR(g.price, 'FM999,999,999.00'), '-') AS price,
        COALESCE(TO_CHAR(g.revenue, 'FM999,999,999.00'), '-') AS revenue,
        COALESCE(TO_CHAR(g.avg_playtime, 'FM999,999,999.00'), '-') AS avg_playtime,
        COALESCE(g.review_score::TEXT, '-') AS review_score,
        COALESCE(p.name, '-') AS publisher_name,
        COALESCE(pc.class, '-') AS publisher_class,
        COALESCE(d.name, '-') AS developer_name
    FROM 
        ivan_patakin.games g
    LEFT JOIN 
        ivan_patakin.publishers p ON g.publisher_id = p.id
    LEFT JOIN 
        ivan_patakin.publisher_class pc ON g.class_id = pc.id
    LEFT JOIN 
        ivan_patakin.developers d ON g.developer_id = d.id;
""")
connection.commit()

print("Создано View для game")

Создано View для game


In [288]:
cursor.execute(""" SELECT * FROM ivan_patakin.games_view ORDER BY steam_id; """)

rows = cursor.fetchall()
headers = ["steam_id", "Название", "Дата релиза", "Проданные копии", "Цена", "Доход", "Среднее время игры", "Оценка", "Издатель", "Класс", "Разработчик"]

print(tabulate(
    rows,
    headers=headers,
    tablefmt="grid",
    colalign=("right", "left", "right", "right", "right", "right", "right", "right", "left", "left", "left"),
    maxcolwidths=[20, 20, 15, 20, 10, 20, 15, 10, 20, 10, 20]
))

+------------+----------------------+---------------+-------------------+--------+----------------+----------------------+----------+----------------------+----------+----------------------+
|   steam_id | Название             |   Дата релиза |   Проданные копии |   Цена |          Доход |   Среднее время игры |   Оценка | Издатель             | Класс    | Разработчик          |
+============+======================+===============+===================+========+================+======================+==========+======================+==========+======================+
|      24880 | The Saboteur™        |    2024-03-07 |         53,376.00 |   9.99 |     290,153.00 |                12.83 |       92 | Electronic Arts      | AAA      | Pandemic Studios     |
+------------+----------------------+---------------+-------------------+--------+----------------+----------------------+----------+----------------------+----------+----------------------+
|     251570 | 7 Days to Die        |    2024

## Удаление таблиц

In [289]:
cursor.execute("""
    DROP VIEW IF EXISTS  ivan_patakin.games_view;
    DROP TABLE IF EXISTS ivan_patakin.games;
    DROP TABLE IF EXISTS ivan_patakin.developers;
    DROP TABLE IF EXISTS ivan_patakin.publisher_class;
    DROP TABLE IF EXISTS ivan_patakin.publishers;
""")

connection.commit()

print("Таблицы удалены")

Таблицы удалены


### Закрытие курсора и соединения

In [290]:
cursor.close()
connection.close()

print("Соединение с PostgreSQL закрыто.")

Соединение с PostgreSQL закрыто.
